In [1]:
import os
data_path = '/private/home/jeanremi/data/crcns/'


def download(file, target='./'):
    import requests

    URL = 'https://portal.nersc.gov/project/crcns/download/index.php'
    
    login_data = dict(
        username='jrking',
        password='pXyh^8*5LzIf',
        fn=file,
        submit='Login' 
        )

    with requests.Session() as s:
        local_filename = login_data['fn'].split('/')[-1]
        r = s.post(URL,data=login_data,stream=True)
        with open(target + local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        print(local_filename)

def download_files():
    files = (
        'vim-2/VoxelResponses_subject1.tar.gz',
        'vim-2/VoxelResponses_subject2.tar.gz',
        'vim-2/VoxelResponses_subject3.tar.gz',
        'vim-2/anatomy.zip',
        'vim-2/Stimuli.tar.gz',
        'vim-2/docs/crcns-vim-2-data-description.docx',
        'vim-2/docs/crcns-vim-2-data-description.pdf',
    )
    for file in files:
        if os.path.isfile(data_path + file.split('/')[-1]):
            continue
        print(data_path + file)
        download(file, data_path)

download_files()

In [2]:
%matplotlib inline

import h5py
import sys
import numpy as np
import cortex
zscore = lambda d: (d-d.mean(0))/d.std(0)

In [4]:
cortex.db.get_mask?

In [3]:
data = []
vdata = []
numSubjects = 3
# subjects is a list of subject names in Pycortex corresponding to the three subjects in the analysis.
subjects = ['S1', 'S2', 'S3']
# xfms is a list of transform names in Pycortex aligning the functional and anatomical data for each subject.
xfms = ['S1_xfm', 'S2_xfm', 'S3_xfm']
dataPath ="./data/VoxelResponses_subject%d.mat"
dataPath = data_path + 'VoxelResponses_subject%d.mat'
for subj in range(numSubjects):
    # Open data file
    f = h5py.File(dataPath % (subj+1),'r')
    # Get size of the data
    datasize = (int(f["ei"]["datasize"].value[2]),int(f["ei"]["datasize"].value[1]),int(f["ei"]["datasize"].value[0]))
    # Get the cortical mask from Pycortex
    mask = cortex.db.get_mask(subjects[subj], xfms[subj], type = 'thick')
    # Get the training data for the subject
    data_subj = np.nan_to_num(zscore(np.nan_to_num(f["rt"].value.T)))
    data.append(data_subj.reshape((data_subj.shape[0],)+datasize)[:, mask])
    # Get the validation data for the subject
    vdata_subj = np.nan_to_num(zscore(np.nan_to_num(f["rv"].value.T)))
    vdata.append(vdata_subj.reshape((vdata_subj.shape[0],)+datasize)[:, mask])

/private/home/jeanremi/.conda/envs/mous/lib/python3.6/site-packages/ipykernel_launcher.py:14: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  


FileNotFoundError: [Errno 2] No such file or directory: 'build/bdist.linux-x86_64/wheel/pycortex-1.0.2.data/data/share/pycortex/db'

In [5]:
debug

> /private/home/jeanremi/.local/lib/python3.6/site-packages/cortex/database.py(178)subjects()
    176         if self._subjects is not None:
    177             return self._subjects
--> 178         subjs = os.listdir(os.path.join(self.filestore))
    179         subjs = [s for s in subjs if os.path.isdir(os.path.join(self.filestore, s))]
    180         self._subjects = dict([(sname, SubjectDB(sname, filestore=self.filestore)) for sname in subjs])

ipdb> self.filestore
'build/bdist.linux-x86_64/wheel/pycortex-1.0.2.data/data/share/pycortex/db'
ipdb> pwd
*** NameError: name 'pwd' is not defined
ipdb> q


In [6]:
subjects[subj]

'S1'

In [7]:
cortex.db

FileNotFoundError: [Errno 2] No such file or directory: 'build/bdist.linux-x86_64/wheel/pycortex-1.0.2.data/data/share/pycortex/db'

In [9]:
pip uninstall pycortex -y

Uninstalling pycortex-1.1.dev0:
  Successfully uninstalled pycortex-1.1.dev0
ERROR: Exception:
Traceback (most recent call last):
  File "/private/home/jeanremi/.conda/envs/mous/lib/python3.6/site-packages/pip/_internal/cli/base_command.py", line 178, in main
    status = self.run(options, args)
  File "/private/home/jeanremi/.conda/envs/mous/lib/python3.6/site-packages/pip/_internal/commands/uninstall.py", line 78, in run
    uninstall_pathset.commit()
  File "/private/home/jeanremi/.conda/envs/mous/lib/python3.6/site-packages/pip/_internal/req/req_uninstall.py", line 444, in commit
    self._moved_paths.commit()
  File "/private/home/jeanremi/.conda/envs/mous/lib/python3.6/site-packages/pip/_internal/req/req_uninstall.py", line 284, in commit
    save_dir.cleanup()
  File "/private/home/jeanremi/.conda/envs/mous/lib/python3.6/site-packages/pip/_internal/utils/temp_dir.py", line 83, in cleanup
    rmtree(self.path)
  File "/private/home/jeanremi/.conda/envs/mous/lib/python3.6/site-pac

In [11]:
pip install -U git+git://github.com/gallantlab/pycortex.git --user

Looking in indexes: https://pypi.org/simple, http://100.97.64.150
  Cloning git://github.com/gallantlab/pycortex.git to /tmp/pip-req-build-cfsvs2ij
  Running command git clone -q git://github.com/gallantlab/pycortex.git /tmp/pip-req-build-cfsvs2ij
  Stored in directory: /tmp/pip-ephem-wheel-cache-zup_qpoi/wheels/e1/21/b1/f4fcf6c8353132b576f6117f3bfd7859e865ae59538299fc36
Successfully built pycortex
Note: you may need to restart the kernel to use updated packages.
